# LangGraph 体验

### Chapter 2: LangGraph 高级特性

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

LLM_NAME = os.getenv("ZHIPU_LLM_NAME") or None
API_BASE = os.getenv("ZHIPU_API_BASE") or None
API_KEY = os.getenv("ZHIPU_API_KEY") or None

### 1. 中断

在 LangGraph 中，我们可以在工作流中设置“断点”，则执行到某一步时，数据流会停止，直至我们再次启动。这样的设计可以很方便地帮助我们控制 AI 的执行走向。

先让我们复制上一章的代码，逐步改造这个简单的 Agent 吧！

In [ ]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]


memory = MemorySaver()

graph_builder = StateGraph(State)

wrapper = DuckDuckGoSearchAPIWrapper(max_results=5)
tool = DuckDuckGoSearchRun(api_wrapper=wrapper)
tools = [tool]
llm = ChatOpenAI(model=LLM_NAME, openai_api_key=API_KEY, openai_api_base=API_BASE)
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(checkpointer=memory, interrupt_before=["tools"])

这段代码绝大部分都与上一章的代码完全一致，但在最后一句上有些出入，多了一句 `interrupt_before=["tools"]`。这句话的意思是，在执行到 `tools` 这个节点之前，会发生中断，工作流会停止下来。（对应的，我们也可以设定 `interrupt_after`）

值得注意的是，LangGraph 中都会以你自己最初设定的环节的名字作为后续工作的代称。

现在再次让我们向构造的 Agent 发问，看看会发生什么！

In [ ]:
user_input = "I'm learning LangGraph. Could you do some research on it for me?"
config = {"configurable": {"thread_id": "1"}}
# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

通过运行上述代码，我们会发现，对于添加了断点的 Agent，用户输入了一句话后，从输出结果中存储的消息记录可看出，整个 Agent 的工作流停留在了 AI 欲调用 Agent 之后，正式调用工具之前。

前文也提到，“状态”是一直跟随着工作流的，所以我们当然可以在工作流中通过 `graph.get_state(config)` 来获取当前的状态。如下文。

In [ ]:
snapshot = graph.get_state(config)
print(f"next node: {snapshot.next}")
existing_message = snapshot.values["messages"][-1]
existing_message.pretty_print()

我们可以看到，状态中的 `messages` 最后一条记录就是工具调用，即还未来得及发出的工具请求。

### 2. 状态添加 / 工作流变更

既然能从工作流中获取到状态、打印状态，那么自然也能够修改状态！

在 LangGraph 中，我们能够任意地修改数据流中的状态，以便于实现更多高度定制化的操作。在下文，我们将尝试“造假”工具请求的结果和 LLM 的回复。

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

answer = (
    "LangGraph is a library for building stateful, multi-actor applications with LLMs."
)
new_messages = [
    # The LLM API expects some ToolMessage to match its tool call. We'll satisfy that here.
    ToolMessage(content=answer, tool_call_id=existing_message.tool_calls[0]["id"]),
    # And then directly "put words in the LLM's mouth" by populating its response.
    AIMessage(content=answer),
]

new_messages[-1].pretty_print()
graph.update_state(
    # Which state to update
    config,
    # The updated values to provide. The messages in our `State` are "append-only", meaning this will be appended
    # to the existing state. We will review how to update existing messages in the next section!
    {"messages": new_messages},
)

print("\n\nLast 2 messages;")
print(graph.get_state(config).values["messages"][-2:])

上文的“造假”过程直接是用的死数据，这个数据来源当然是可以来源于外界，例如用户、一个YOLO、AC自动机或专门用于系统整体监督的另一个 Agent。

上文的“造假”过程一次性造假了工具的请求回复信息和 LLM 的回答，而我们也知道，LLM 的回答即代表 `__end__`，即工作流的结束。我们可以在此时再次查看 LangGraph 当前内部的状态情况，如下文。

In [ ]:
snapshot = graph.get_state(config)
print(snapshot.values["messages"][-3:])
print(snapshot.next)

这样的输出记录的改变会影响工作流的改变，而这些改变都可以被 LangGraph 感知。

从上面的输出也可看出，`snapshot.next`的输出为空（他下一步没有要做的事情了！），所以我们可以通过这种方式，借助外部因素控制工作流的走向。

### 3. 状态更新 / 内容更正

假定当前有一个需求，我们需要对消息记录中的某条信息进行修正，例如调用工具时输入的参数、工具的结果返回等等。LangGraph 也提供了这样的功能，我们可以通过 `graph.update_state` 来实现这一需求。

如下文，我们先重新启用一次对话吧！

In [ ]:
user_input = "I'm learning LangGraph. Could you do some web research on it for me?"
config = {"configurable": {"thread_id": "2"}}  # 重新开始一个 id 为 2 的对话
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

依照前面制定的 graph，这个对话会在调用工具之前中断。我们可以通过 `graph.get_state(config)` 来查看当前的状态。

In [ ]:
snapshot = graph.get_state(config)
existing_message = snapshot.values["messages"][-1]
print("Original")
print("Message ID", existing_message.id)
print(existing_message.tool_calls[0])

我们可以观察到，LLM 再次卡在了正式调用工具前，当前只提供了调用工具要用到的输入参数，而没有真正执行工具调用的过程。

则此时我们可以插手，修改 LLM 调用工具打算输入的参数。如下文。

In [ ]:
from langchain_core.messages import AIMessage

new_tool_call = existing_message.tool_calls[0].copy()
new_tool_call["args"]["query"] = "LangGraph human-in-the-loop workflow"
new_message = AIMessage(
    content=existing_message.content,
    tool_calls=[new_tool_call],
    # Important! The ID is how LangGraph knows to REPLACE the message in the state rather than APPEND this messages
    id=existing_message.id,
)

print("Updated")
print(new_message.tool_calls[0])
print("Message ID", new_message.id)
graph.update_state(config, {"messages": [new_message]})

print("\n\nTool calls")
graph.get_state(config).values["messages"][-1].tool_calls

需要注意的是，要修改掉原有的记录，我们构建的新信息的 id 必须和原信息完全一致，此时再次查看状态，我们会发现，原有的信息已经被新信息替代了。

此时我们再次调用 stream 或 invoke 方法，第一个参数不再提供任何内容，则工作流会继续执行，直至结束。如下文。

In [ ]:
events = graph.stream(None, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

带上我们的用户对话标识 config，检查对话上下文是否被正确保存，如下文。

In [ ]:
events = graph.stream(
    {
        "messages": (
            "user",
            "Remember what I'm learning about?",
        )
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

LLM 记住了历史对话并正确总结回答了！